In [1]:
import load
import aggregate

import aliases
from everest import window

In [2]:
mydata = load.FBDataset('vic')
spatialaggregator = aggregate.SpatialAggregator('lga')

In [3]:
sample = mydata[-2:]

Loading many files from 'vic'

Aggregating tiles...
Tiles aggregated.

Done.


In [4]:
frm = aggregate.aggregate(sample)

Aggregating to lga...
Adding possible journeys...
Getting quadkey weights...
Quadkey weights retrieved.
Added possible journeys.
Splitting frame by possible journey...
Frame split.
Aggregated.
Combining dates...
Dates combined.


In [7]:
frm

km   n  weight      time
date       start stop                                  
2021-06-22 49399 49399   0.000000   7     1.0  18:00:00
           40520 40520   0.000000   6     1.0  18:00:00
                 46670  14.569124   8     1.0  18:00:00
                 46670   4.038133   6     1.0  18:00:00
           43790 43790   0.000000  33     1.0  18:00:00
...                           ...  ..     ...       ...
2021-06-23 10550 10550   3.906906   8     1.0  02:00:00
                 10550  21.078068   6     1.0  02:00:00
                 10550  12.343332   7     1.0  02:00:00
                 10550   8.760227   2     1.0  02:00:00
                 10550  12.388832   6     1.0  02:00:00

[37213 rows x 4 columns]

In [6]:
frm.groupby(level = 'date')

In [ ]:
mydata.timezone

In [ ]:
sample

In [ ]:
datetimes = sample.reset_index()['datetime']

In [ ]:
datetimes.dt.tz_convert(mydata.timezone)

In [ ]:
sample

In [ ]:
dates = datetimes.apply(lambda x: x.date)

In [ ]:
dates

In [ ]:
frm.groupby('d')

In [ ]:
def combine_by_date(frm):
    indexnames = frm.index.names
    frm = frm.reset_index()
    frm['date'] = frm['datetime'].apply(lambda x: x.date)
    frm['time'] = frm['datetime'].apply(lambda x: x.time)
    frm = frm.drop('datetime', axis = 1)
    frm = frm.set_index(['date', *indexnames[1:]])
    return frm

In [ ]:
combine_by_date(frm)

In [ ]:
frm.reset_index()['datetime'].apply(lambda x: x.date)

In [ ]:
import pandas as pd

def make_date(d):
    return '-'.join([str(x).zfill(2) for x in d.timetuple()[:3]])
def aggregate_by_date(
        frm,
        datetimeKey = 'datetime',
        ):
    print("Aggregating by date...")
    frm = frm.copy()
    frm['date'] = list(frm.reset_index()[datetimeKey].apply(make_date))
    frm['date'] = pd.to_datetime(frm['date']).dt.date
    indexNames = ['date' if nm == datetimeKey else nm for nm in frm.index.names]
    frm = frm.reset_index().set_index(indexNames)
    frm = frm.sort_index()
    frm = frm.drop('datetime', axis = 1)
    print("Aggregated.")
    return frm

In [ ]:
out = aggregate_by_date(frm)

In [ ]:
out

In [ ]:
frm.groupby(level = 'datetime').aggregate(
#     lambda x: x.loc[x['km'] == 0]['n'].sum() / x['n'].sum()
    lambda x: print(type(x))
    )

In [ ]:
frm.groupby(level = ('datetime', 'start', 'stop'))

In [ ]:
savefrm = frm

In [ ]:
import numpy as np
import pandas as pd

frm = frm.reset_index()
lens = [len(item) for item in frm['possible_journeys']]
journeys = [
    arr.flatten()
        for arr in np.split(
            np.concatenate(frm['possible_journeys'].values),
            3,
            axis = 1
            )
    ]
frm = pd.DataFrame({
    **{name: np.repeat(frm[name].values, lens) for name in frm.columns}, 
    **{name: val for name, val in zip(['start', 'stop', 'weight'], journeys)}
    })
frm['weight'] = frm['weight'].astype(float)
dropKeys = {'quadkey', 'end_key', 'index', 'possible_journeys'}
frm = frm.drop(dropKeys, axis = 1)
frm['start'], frm['stop'] = frm['start'].astype(int), frm['stop'].astype(int)
frm = frm.set_index(['datetime', 'start', 'stop'])

In [ ]:


test = pd.DataFrame({
    **{name: np.repeat(frm[name].values, lens) for name in frm.columns}, 
    **{name: val for name, val in zip(['start', 'stop', 'weight'], journeys)}
    })

In [ ]:
from itertools import product
import numpy as np

import pandas as pd
df = pd.DataFrame
import geopandas as gpd
gdf = gpd.GeoDataFrame
sjoin = gpd.tools.sjoin

# from get import get_quadFrm, get_intersection_weights
from utils import quadkeys_to_poly

def aggregate_mob_tiles_to_regions(frm, aggtype = 'lga'):

#     print("Aggregating from tiles to regions...")

#     # Trim frame
#     frm = frm.copy()
#     indexnames = frm.index.names
#     frm = frm.reset_index()
#     quadkeys = [*frm['quadkey'], *frm['end_key']]
#     bounds = quadkeys_to_poly(quadkeys).envelope
#     toFrm = toFrm.loc[toFrm.intersects(bounds)]
#     if weights is None:
#         quadFrm = get_quadFrm(quadkeys)
#         weights = get_intersection_weights(quadFrm, toFrm)
#     frm = frm.reset_index().set_index('quadkey')
#     frm = frm.drop(
#         set(frm.index).difference(set(weights.keys()))
#         )
#     frm = frm.reset_index().set_index('end_key')
#     frm = frm.drop(
#         set(frm.index).difference(set(weights.keys()))
#         )
#     frm = frm.reset_index().set_index(indexNames)

#     def group_func(x):
#         startWeights, endWeights = x[['start_weights', 'end_weights']].values[0]
#         possibleJourneys = list(product(startWeights, endWeights))
#         outRows = []
#         for pair in possibleJourneys:
#             (start, startWeight), (end, endWeight) = pair
#             outRow = [int(start), int(end), startWeight * endWeight]
#             outRows.append(outRow)
#         return outRows

#     indexNames = frm.index.names
#     frm = frm.reset_index()
#     frm['start_weights'] = frm.reset_index()['quadkey'].apply(
#         lambda x: weights[x]
#         )
#     frm['end_weights'] = frm.reset_index()['end_key'].apply(
#         lambda x: weights[x]
#         )
#     groupby = frm.groupby(['quadkey', 'end_key'])
#     groupby = groupby[['start_weights', 'end_weights']]
#     frm = frm.reset_index().set_index(['quadkey', 'end_key'])
#     frm['possible_journeys'] = groupby.apply(group_func)
#     frm = frm.reset_index().set_index(indexNames)
#     frm = frm.drop({'start_weights', 'end_weights', 'index'}, axis = 1)

    frm = frm.reset_index()
    lens = [len(item) for item in frm['possible_journeys']]
    journeys = [
        arr.flatten()
            for arr in np.split(
                np.concatenate(frm['possible_journeys'].values),
                3,
                axis = 1
                )
        ]
    frm = df({
        **{name: np.repeat(frm[name].values, lens) for name in frm.columns}, 
        **{name: val for name, val in zip(['start', 'stop', 'weight'], journeys)}
        })
    frm['weight'] = frm['weight'].astype(float)
    dropKeys = {'quadkey', 'end_key', 'level_0', 'possible_journeys'}
    frm = frm.drop(dropKeys, axis = 1)
    frm['start'], frm['stop'] = frm['start'].astype(int), frm['stop'].astype(int)
    frm = frm.set_index(['datetime', 'start', 'stop'])

    buffer = bounds.buffer(np.sqrt(bounds.area) * 0.1)
    indexNames = frm.index.names
    clippedFrm = toFrm.loc[toFrm.within(buffer)]
    frm = frm.reset_index().set_index('start')
    frm = frm.drop(set(frm.index).difference(set(clippedFrm.index)))
    frm = frm.reset_index().set_index(indexNames)

    frm = frm.sort_index()

    print("Aggregated.")

    return frm

In [ ]:
lgas = load.load_lgas()

In [ ]:
sample

In [ ]:
lgas[quad]

In [ ]:
quadweights

In [ ]:
l